In [1]:
import findspark
findspark.init('/home/fede/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [10]:
from pyspark.ml.regression import LinearRegression

In [12]:
file_path = '/home/fede/sample_linear_regression_data.txt'
training = spark.read.format('libsvm').load(file_path)

In [13]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [15]:
lr = LinearRegression(featuresCol='features',labelCol='label',
                     predictionCol='prediction')

In [16]:
lrModel = lr.fit(training)

In [17]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [18]:
lrModel.intercept

0.14228558260358093

In [19]:
training_summary = lrModel.summary

In [20]:
training_summary.r2

0.027839179518600154

In [21]:
training_summary.rootMeanSquaredError

10.16309157133015

In [22]:
all_data = spark.read.format('libsvm').load(file_path)

In [26]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [28]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                361|
|   mean|0.14268765306943054|
| stddev| 10.345347205831674|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [32]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                140|
|   mean| 0.5513646940116145|
| stddev| 10.277824801592072|
|    min|-23.487440120936512|
|    max| 27.111027963108548|
+-------+-------------------+



In [33]:
correct_model = lr.fit(train_data)

In [34]:
test_results = correct_model.evaluate(test_data)

In [35]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-23.567485360065586|
| -25.89550201300154|
|-19.308691723409297|
|-21.357217662514756|
|-19.735609775232028|
| -19.79287321597993|
|-20.471546755869223|
|-19.798610501140594|
|-17.536960004104102|
|-15.884041054263188|
| -16.54585522167062|
|-15.678367103058406|
|-17.595049940978324|
|-17.300804570868618|
|-17.159030143439146|
|-14.379826057327909|
| -9.890645304654903|
|-15.719083021700584|
| -8.783243724829662|
|-13.796986323199732|
+-------------------+
only showing top 20 rows



In [36]:
test_results.rootMeanSquaredError

10.365214937516212

In [37]:
unlabeled_data = test_data.select('features')

In [38]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [39]:
predictions = correct_model.transform(unlabeled_data)

In [40]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| 0.08004523912907416|
|(10,[0,1,2,3,4,5,...|  2.9456760768054675|
|(10,[0,1,2,3,4,5,...| -0.5758690508641284|
|(10,[0,1,2,3,4,5,...|  1.5744548729002181|
|(10,[0,1,2,3,4,5,...| 0.06829115986030995|
|(10,[0,1,2,3,4,5,...|  0.3905371857653782|
|(10,[0,1,2,3,4,5,...|  2.1963331898645926|
|(10,[0,1,2,3,4,5,...|  2.3699359302010867|
|(10,[0,1,2,3,4,5,...|  0.2102392714281557|
|(10,[0,1,2,3,4,5,...| -0.8350557793419021|
|(10,[0,1,2,3,4,5,...| -0.1463517996404862|
|(10,[0,1,2,3,4,5,...|-0.40729193796308416|
|(10,[0,1,2,3,4,5,...|  1.7330406134077636|
|(10,[0,1,2,3,4,5,...|  1.5687162986293735|
|(10,[0,1,2,3,4,5,...|   1.783172420126848|
|(10,[0,1,2,3,4,5,...|  0.5127381621691413|
|(10,[0,1,2,3,4,5,...| -3.2626903017106272|
|(10,[0,1,2,3,4,5,...|  3.2276409441541705|
|(10,[0,1,2,3,4,5,...| -3.6277026780104986|
|(10,[0,1,2,3,4,5,...|  1.598889